<a href="https://colab.research.google.com/github/sankalp294/Combat-Solutions-tasks/blob/main/5_DAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langgraph langchain-core

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
import operator
from typing import Annotated

print(" Setup complete!\n")


class GraphState(TypedDict):
    user_input: str
    messages: list
    route: str
    result: str


def router_node(state: GraphState) -> GraphState:
    user_input = state["user_input"]

    # Check if it's a calculation query
    calc_keywords = ['calculate', 'compute', 'what is', 'solve']
    math_operators = ['+', '-', '*', '/']

    if any(keyword in user_input.lower() for keyword in calc_keywords) or \
       any(op in user_input for op in math_operators):
        state["route"] = "calculator"
    else:
        state["route"] = "llm"

    state["messages"] = [f"Router → {state['route'].upper()}"]  # Replace instead of append
    return state


def calculator_node(state: GraphState) -> GraphState:
    user_input = state["user_input"]

    try:

        import re
        match = re.search(r'(\d+\s*[\+\-\*/]\s*\d+(?:\s*[\+\-\*/]\s*\d+)*)', user_input)

        if match:
            expr = match.group(1)
            result = eval(expr)
            state["result"] = f"Calculator: {expr} = {result}"
        else:
            state["result"] = "Calculator: No expression found"
    except Exception as e:
        state["result"] = f"Calculator: Error - {str(e)}"

    state["messages"].append(state["result"])
    return state


def llm_node(state: GraphState) -> GraphState:
    user_input = state["user_input"].lower()

    if 'hello' in user_input or 'hi' in user_input:
        response = "Hi! How can I help?"
    elif 'how are you' in user_input:
        response = "I'm great, thanks!"
    else:
        response = f"You asked: '{state['user_input']}'"

    state["result"] = f"LLM: {response}"
    state["messages"].append(state["result"])
    return state

def route_decision(state: GraphState) -> Literal["calculator", "llm"]:
    return state["route"]

workflow = StateGraph(GraphState)

workflow.add_node("router", router_node)
workflow.add_node("calculator", calculator_node)
workflow.add_node("llm", llm_node)

workflow.set_entry_point("router")

workflow.add_conditional_edges(
    "router",
    route_decision,
    {"calculator": "calculator", "llm": "llm"}
)

workflow.add_edge("calculator", END)
workflow.add_edge("llm", END)

app = workflow.compile()

print(" DAG built!\n")
print("Graph: START → router → [calculator | llm] → END\n")

def test(query: str):
    print(f"\n{'='*50}")
    print(f"Query: {query}")
    print('='*50)

    state = {
        "user_input": query,
        "messages": [],
        "route": "",
        "result": ""
    }

    result = app.invoke(state)

    print(f"Route: {result['route']}")
    print(f"Result: {result['result']}")
    print("Messages:")
    for msg in result['messages']:
        print(f"  - {msg}")

print(" Testing both routes:\n")

test("Calculate 25 * 4 + 10")
test("Hello, how are you?")
test("What is 100 / 5?")
test("What can you do?")
test("45 + 55")

print("\n\nDone! Both routes tested successfully!")


 Setup complete!

 DAG built!

Graph: START → router → [calculator | llm] → END

 Testing both routes:


Query: Calculate 25 * 4 + 10
Route: calculator
Result: Calculator: 25 * 4 + 10 = 110
Messages:
  - Router → CALCULATOR
  - Calculator: 25 * 4 + 10 = 110

Query: Hello, how are you?
Route: llm
Result: LLM: Hi! How can I help?
Messages:
  - Router → LLM
  - LLM: Hi! How can I help?

Query: What is 100 / 5?
Route: calculator
Result: Calculator: 100 / 5 = 20.0
Messages:
  - Router → CALCULATOR
  - Calculator: 100 / 5 = 20.0

Query: What can you do?
Route: llm
Result: LLM: You asked: 'What can you do?'
Messages:
  - Router → LLM
  - LLM: You asked: 'What can you do?'

Query: 45 + 55
Route: calculator
Result: Calculator: 45 + 55 = 100
Messages:
  - Router → CALCULATOR
  - Calculator: 45 + 55 = 100


Done! Both routes tested successfully!
